In [31]:
import cv2
import glob
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
import numpy as np
import os
import re
from scipy import spatial
from scipy import signal
from scipy import ndimage
from scipy import optimize

%matplotlib

Using matplotlib backend: QtAgg


In [32]:
def xcorr_prep(im):
    im_corr = np.sum(im.astype(np.float64), axis=2) # ensure single channel
    im_corr = np.max(im_corr) - im_corr # invert value
    im_corr /= im_corr.max() # normalize
    im_corr -= np.median(im_corr) # detrend
    return im_corr

In [33]:
def gen_template(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)

    pattern_size = 8
    pattern = np.random.choice([0.1,0.9], size=(pattern_size,pattern_size))
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm

    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='silver')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='silver')) # vert
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 4), c_size, c_thick / 2, color='k')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 4, h_px / 2 - c_size / 2), c_thick / 2, c_size, color='k')) # vert

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

In [34]:
np.random.seed(77777) # generate same noise pattern every time
gen_locs = ['SW', 'NW', 'NE', 'SE', 'raft']
TARGET_W = 0.03 # 40 mm
TARGET_H = TARGET_W
for loc in gen_locs:
    gen_template(TARGET_W, TARGET_H, 300, loc, plot=False)

In [38]:
plt.close('all')

# https://learnopencv.com/camera-calibration-using-opencv/
CHECKERBOARD = (5,8)
objpoints = []
imgpoints = []
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
square_size = 0.025 # 25 mm
objp *= square_size
prev_img_shape = None
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for fname in sorted(glob.glob('input/**.*', recursive=True)):
    # do camera calibration from chessboard images
    im = image.imread(fname)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    print(ret)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (5,5),(-1,-1), criteria)
        imgpoints.append(corners2)
        im = cv2.drawChessboardCorners(im, CHECKERBOARD, corners2, ret)
    plt.figure()
    plt.imshow(im)
    ax = plt.gca()
    ax.set_aspect('equal')

h,w = im.shape[:2]
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
optimal_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
    mtx,
    dist, 
    (w,h), 
    1, 
    (w,h)
)
undistorted_image = cv2.undistort(
    im,
    mtx,
    dist,
    None,
    optimal_camera_matrix
)
print(tvecs)
plt.imshow(undistorted_image)
ax = plt.gca()
ax.set_aspect('equal')


True
True
True
True
True
True


In [36]:
plt.close('all')

chessboard_sq_side = 0.025 # m

px_per_ms = []
files = sorted(glob.glob('input/**.jpg', recursive=True))
# which image will we deproject all others to be like?
ref_idx = len(files) - 1
print(files)
for img_idx in range(len(files)):
    plt.figure()
    ax = plt.axes()
    ax.set_aspect('equal')
    ax.grid(True)

    # use camera cal matrix to de-distort all images
    im = image.imread(files[img_idx])
    undistorted_image = cv2.undistort(
        im,
        mtx,
        dist,
        None, 
        optimal_camera_matrix
    )

    this_obj = objpoints[img_idx][0,:,:2]
    this_img = imgpoints[img_idx][:,0,:]

    h, status = cv2.findHomography(
        this_img,
        imgpoints[ref_idx][:,0,:]
    )
    im_warped = cv2.warpPerspective(
        undistorted_image,
        h,
        (undistorted_image.shape[1], undistorted_image.shape[0])
    )
    gray = cv2.cvtColor(im_warped, cv2.COLOR_BGR2GRAY)
    # ax.imshow(im_warped)

    # locate chessboard corners in de-distorted, deprojected image to calcualte pixel scale
    ret, corners = cv2.findChessboardCorners(
        gray,
        CHECKERBOARD,
        cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE
    )
    if ret == True:
        corners2 = cv2.cornerSubPix(gray, corners, (5,5),(-1,-1), criteria)
        im_warped_corners = cv2.drawChessboardCorners(im_warped, CHECKERBOARD, corners2, ret)
        im_warped_corners_grey = cv2.cvtColor(im_warped_corners, cv2.COLOR_BGR2GRAY)
    cmap = 'Greys'
    if img_idx == ref_idx:
        cmap = 'viridis'
    ax.imshow(im_warped_corners_grey, cmap=cmap)

    # calc pixel scale: get pairs of euclidean distances along each axis
    found_corners = corners2[:,0,:].reshape((CHECKERBOARD[1], CHECKERBOARD[0], 2))
    dists = np.array([])
    for i in range(found_corners.shape[0]):
        d = np.diff(found_corners[i,:,:], axis=0)
        dists = np.concatenate([dists, np.sqrt((d ** 2).sum(axis=1))])
    for j in range(found_corners.shape[1]):
        d = np.diff(found_corners[:,j,:], axis=0)
        dists = np.concatenate([dists, np.sqrt((d ** 2).sum(axis=1))])
    avg_spacing = np.mean(np.abs(dists))
    px_per_m = avg_spacing / chessboard_sq_side
    px_per_ms.append(px_per_m)
    print('px per m', px_per_m)

frac_err = np.std(px_per_ms) / np.mean(px_per_ms)
avg_px_per_m = np.mean(px_per_ms)
print('std error induced by pixel cal across span of .7m:', 0.7 * frac_err)

['input/IMG_6928.jpg', 'input/IMG_6929.jpg', 'input/IMG_6930.jpg', 'input/IMG_6931.jpg', 'input/IMG_6932.jpg', 'input/IMG_6933.jpg', 'input/IMG_6934.jpg', 'input/IMG_6935.jpg', 'input/IMG_6936.jpg']
px per m 2276.214039361299
px per m 2251.649588969216
px per m 2622.1913374715773
px per m 2774.5861656986067
px per m 3029.411438899254
px per m 3126.256802687004
px per m 2369.0549935867534
px per m 2299.1099161176535
px per m 3034.367965015013
std error induced by pixel cal across span of .7m: 0.08982044030763624


In [37]:
plt.close('all')
tmp = ndimage.zoom(image.imread('input\\raft_crop.jpeg'), 2)
stride = 2
a = xcorr_prep(im_warped)[::stride,::stride]
b = xcorr_prep(tmp)[::stride,::stride]
f1 = plt.figure()
plt.imshow(a)
f2 = plt.figure()
plt.imshow(b)
xcorr = signal.fftconvolve(
    a,
    b[::-1, ::-1],
    mode='valid'
)
f3 = plt.figure()
plt.imshow(xcorr)
y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
xfound = x + tmp.shape[0] / 2
yfound = y + tmp.shape[1] / 2
ax = f1.get_axes()[0]
ax.scatter(xfound, yfound)


FileNotFoundError: [Errno 2] No such file or directory: 'input\\raft_crop.jpeg'

##### Plot location of each point

In [ ]:
# plt.close('all')

# plt.figure()
# ax = plt.axes()
# ax.set_aspect('equal')
# order = 0
# found_corners = corners2[:,0,:].reshape((CHECKERBOARD[1], CHECKERBOARD[0], 2))
# for i in range(found_corners.shape[0]):
#     for j in range(found_corners.shape[1]):
#         ax.scatter(found_corners[i][j][0], found_corners[i][j][1], color='g')
#         ax.annotate(f'{order}', (found_corners[i][j][0], found_corners[i][j][1]), fontsize=12)
#         order += 1
# # calc pixel scale: get pairs of euclidean distances along each axis
# dists = np.array([])
# for i in range(found_corners.shape[0]):
#     d = np.diff(found_corners[i,:,:], axis=0)
#     dists = np.concatenate([dists, np.sqrt((d ** 2).sum(axis=1))])
# for j in range(found_corners.shape[1]):
#     d = np.diff(found_corners[:,j,:], axis=0)
#     dists = np.concatenate([dists, np.sqrt((d ** 2).sum(axis=1))])
# avg_spacing = np.mean(np.abs(dists))
# print('px per m', avg_spacing / 0.02)


px per m 3750.1422512915824
